In [1]:
# %pip install PyMuPDF

In [2]:
# %pip install tiktoken

In [3]:
# %pip install langchain_openai

In [1]:
import re
import os
import config
from glob import glob

from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# 설정 정보 읽기
CHUNK_SIZE = config.chunk_size
CHUNK_OVERLAP = config.chunk_overlap

MODEL_NAME  = config.model_name
EMBEDDING_NAME = config.embedding_name

COLLECTION_NAME = config.collection_name_case
PERSIST_DIRECTORY = config.persist_directory_case


In [3]:
name = {"고대", "고려", "근대", "조선", "현대"}

In [6]:
for i in name:
    path_list = glob(f"pdf/사건/{i}/*.pdf")

    document_list = []

    # load -> 전처리 -> split -> metadata 추가 
    for path in path_list:
        # Document Load
        loader = PyMuPDFLoader(path)
        load_docs = loader.load()

        # 전처리 - 관련사료, 한자 제거
        full_text = [doc.page_content for doc in load_docs] 
        full_text = ''.join(full_text)
        full_text = re.sub(r"관련사료", "", full_text)
        full_text = re.sub(r"\([一-龥]+\)", "", full_text)
        full_text = re.sub(r"\n", "", full_text)

        # Split
        splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            model_name=MODEL_NAME,
            chunk_size=CHUNK_SIZE,
            chunk_overlap=CHUNK_OVERLAP,
        )
        # spiltter.documents([Document, Document, ...]) -> list[Document]
        docs = splitter.split_text(full_text) # 문자열 split -> list[str]

        # Metadata 생성
        title = "_".join(os.path.splitext(os.path.basename(path))[0].split('_')[:2])
        sub_title = os.path.splitext(os.path.basename(path))[0].split('_')[-1]
        metadata = {
            "title":title,
            "sub_title": sub_title,
            # "full_text": full_text
        }

        # Document 생성
        for doc in docs: # docs: split된 소설 내용들. list[str]
            _doc = Document(metadata=metadata, page_content=doc)
            document_list.append(_doc)

    embedding_model = OpenAIEmbeddings(
        model=EMBEDDING_NAME
    )
    # 없으면 생성 / 연결 + document들 추가
    vector_store = Chroma.from_documents(
        documents=document_list,
        embedding=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
    )
    print(len(document_list))

114
175
121


c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 3 of document pdf/사건/조선\사건_조선_한산도해전.pdf
  warnings.warn(


391
335


In [7]:
vector_store._collection.count()

2792

In [7]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

embedding_model = OpenAIEmbeddings(
    model=EMBEDDING_NAME
)
# 없으면 생성 / 연결 + document들 추가
vector_store = Chroma.from_documents(
    documents=document_list,
    embedding=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)
vector_store._collection.count()

692

In [24]:
query = "강화도와 관련된 사건을 알려줘"

result = vector_store.similarity_search_with_score(
    query,
    k=8,
    # filter={"author": "현진건"}
)

In [25]:
result

[(Document(metadata={'sub_title': '강홍립', 'title': '인물_조선'}, page_content='순식간에 평안도를 돌파하여 황해도에 이르렀다. 그러나 후금군 역시 전쟁의 주요 목적인 가도의 모문룡 제거에실패했고, 조선을 완전 정복할 준비가 되어 있지 않았다. 실제로 후금군은 전쟁 초기부터 조선에강화를 먼저 요구해 놓은 상태였다.조선측에서도 강화에 적극 호응하였는데, 이때 강홍립의 존재가 다시 중요해졌다. 조선에서는강홍립과 박난영에게 서신을 보내어 후금의 입장을 탐색하면서 강화의 실마리를 얻고자 했다.후금에서도 강홍립을 강화 주선에 이용하였고, 강홍립은 조선 조정에 편지를 보내어 후금이 강화를 원하고 있다는 소식을 전하였다. 이에 조정에서는 강홍립의 숙부 강인과 아들 강숙을 후금진영에 왕래하게 하여 강화의 사자로 삼았다. 강홍립의 존재가 위기에 빠진 조선으로서는 더없이 귀중해진 것이다. 조선 조정에서는 강홍립과 박난영에게 이전의 허물을 용서하고 후한 상을주겠다고 회유하기까지 했으며, 그 아들들에게는 벼슬을 내렸다.  강홍립은 인조와 조선 조정의 기대에 부응하였다. 2월 9일, 인조가 피난해 있던 강화도로 강홍립과 박난영이 후금의 사신과 함께 찾아왔다. 조정에서는 척화론을 주장한 윤황과 같이 강홍립을 처형해야 한다고 주장하는 사람이 많았으나, 인조는 이를 묵살하고 2월 10일 강홍립과 박5난영을 접견하였다. 강홍립은 그 자리에서 인조에게 후금군이 쳐들어온 이유와 후금군 내부의강화 논의를 자세하게 설명하였다.  강홍립은 조선에 후금에 대한 정보를 알려줌은 물론, 후금군 진영으로 돌아간 후에도 조선 조정과 지속적으로 서신을 왕래하면서 강화의 체결 및 후금군의 약탈 방지를 위해 힘을 썼다. 그 결과 강화가 체결되었고, 강홍립은 조선에 영구 귀국할 수 있었다.그러나 강홍립의 말년은 평탄치 못했다. 인조는 강홍립의 관작을 회복해 주고, 그에게 요미를 지급하는 등 후하게 대우했다. 하지만 강홍립의 어머니는 그가 후금에 억류되어 있는 동안 